In [1]:
import sys
import os
import warnings
import traceback
import logging
import time
import dotenv
dotenv.load_dotenv(".env")

utils_path = "/Users/benminh1201/Downloads/nyc_new/utils"
sys.path.append(utils_path)
from helpers import load_cfg
from minio_utils import MinIOClient

from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder \
            .config("spark.driver.memory", "14g") \
            .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
            .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
            .config("spark.sql.catalog.local.type", "hadoop") \
            .config("spark.sql.catalog.local.warehouse", "s3a://iceberg/staging") \
            .config("spark.sql.parquet.enableVectorizedReader", "false") \
            .config("spark.sql.parquet.writeLegacyFormat", "true") \
            .getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
# Set the MinIO access key, secret key, endpoint, and other configurations
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "benminh1201")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "benminh1201")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

25/03/27 22:33:17 WARN Utils: Your hostname, benminh1201.local resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
25/03/27 22:33:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/27 22:33:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark.catalog.tableExists("local.nyc.taxi")

True

In [6]:
df = spark.read.parquet("s3a://raw/yellow_tripdata/2020/yellow_tripdata_2020-01.parquet")

In [7]:
df.writeTo("local.nyc.taxi").createOrReplace()

25/03/27 22:39:51 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to staging/nyc/taxi/data/00003-4-c2889c9a-acc9-4e1b-98c1-285d15efea72-0-00001.parquet. This is unsupported
25/03/27 22:39:52 WARN HadoopTableOperations: Error reading version hint file s3a://iceberg/staging/nyc/taxi/metadata/version-hint.text
java.io.FileNotFoundException: No such file or directory: s3a://iceberg/staging/nyc/taxi/metadata/version-hint.text
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3866)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.extractOrFetchSimpleFileStatus(S3AFileSystem.java:5401)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.open(S3AFileSystem.java:1465)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.open(S3AFileSystem.java:1441)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:976)
	at org.apache.iceberg.hadoop.HadoopTableOperations.findVe